In [ ]:
!pip install cohere umap-learn altair annoy datasets tqdm bertopic

In [ ]:
!pip install bertopic

In [4]:
#@title Import libraries (Run this cell to execute required code) {display-mode: "form"}

import cohere
import numpy as np
import re
import pandas as pd
from tqdm import tqdm
from datasets import load_dataset
import umap
import altair as alt
from sklearn.metrics.pairwise import cosine_similarity
from annoy import AnnoyIndex
import warnings
from sklearn.cluster import KMeans

from sklearn.feature_extraction.text import CountVectorizer

warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', None)

In [32]:
import pandas as pd
# Load data from CSV into a DataFrame
df = pd.read_csv('/content/drive/MyDrive/ML/reviews_data.csv')

In [30]:
# Get text embeddings
def get_embeddings(texts,model='embed-english-v2.0'):
  output = co.embed(
                model=model,
                texts=texts)
  return output.embeddings

In [ ]:
# Embed the dataset
embeds=pd.DataFrame()
embeds['reviews_embeds'] = get_embeddings(df['reviews'].tolist())
embeds.head()


In [ ]:
embeds.head(1)

In [6]:
# import pandas as pd
# from gensim.models import Word2Vec
# from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences

# # Load data from CSV into a DataFrame
# df = pd.read_csv('/content/drive/MyDrive/ML/reviews_data.csv')

# # Preprocess and tokenize the text data
# tokenizer = Tokenizer()
# tokenizer.fit_on_texts(df['reviews'])
# sequences = tokenizer.texts_to_sequences(df['reviews'])
# max_seq_length = max(len(seq) for seq in sequences)
# vocab_size = len(tokenizer.word_index) + 1

# # Apply Word2Vec embedding technique
# word2vec_model = Word2Vec(sentences=sequences, vector_size=100, window=5, min_count=1, workers=4)

# # Create the embedding matrix
# embedding_dim = 100
# embedding_matrix = np.zeros((vocab_size, embedding_dim))
# for word, i in tokenizer.word_index.items():
#     if word in word2vec_model.wv:
#         embedding_matrix[i] = word2vec_model.wv[word]

# # Use the embedding matrix in your model
# # ...

# # Additional steps:
# # - Pad sequences to a fixed length (if necessary) using pad_sequences()
# # - Split your data into train/test/validation sets
# # - Train your model using the embedding matrix and the padded sequences
# # - Evaluate the model performance


In [8]:
#Load the embeddings matrix
# embeds = embedding_matrix

In [36]:
# embeds=df_embed

In [44]:
embeds.shape

(1494, 1)

In [45]:
print(f'Loaded a DataFrame with {len(df)} rows and an embeddings matrix of dimensions {embeds.shape}')

Loaded a DataFrame with 1494 rows and an embeddings matrix of dimensions (1494, 1)


Building a semantic search index
For nearest-neighbor search, we can use the open-source Annoy library. Let's create a semantic search index and feed it all the embeddings.

In [40]:
print(embeds.shape)

(1494, 1)


In [49]:
print(embeds.head(1))

In [51]:
# # Create the search index, pass the size of embedding
# search_index = AnnoyIndex(embeds.shape[1], 'angular')
# # Add all the vectors to the search index
# for i in range(len(embeds)):
#     search_index.add_item(i, embeds['reviews_embeds'][i])

# search_index.build(10) # 10 trees
# search_index.save('askhn.ann')

IndexError: ignored

In [54]:
from annoy import AnnoyIndex
import numpy as np

# Convert the 'reviews_embeds' column to a numpy array
embeds_array = np.array(embeds['reviews_embeds'].tolist())

# Create the search index, pass the size of the embedding
search_index = AnnoyIndex(embeds_array.shape[1], 'angular')

# Add all the vectors to the search index
for i in range(len(embeds_array)):
    search_index.add_item(i, embeds_array[i])

search_index


1- Given an existing reviews title, retrieve the most similar reviews (nearest neighbor search using embeddings) We can query neighbors of a specific reviews using get_nns_by_item.

In [ ]:
df.head()

In [56]:
# Choose an example (we'll retrieve others similar to it)
example_id = 50

# Retrieve nearest neighbors
similar_item_ids = search_index.get_nns_by_item(example_id,
                                                10,  # Number of results to retrieve
                                                include_distances=True)

# Format and print the text and distances
mask = df.index.isin(similar_item_ids[0]) & (df.index != example_id)
results = pd.DataFrame(data={'reviews': df.loc[mask, 'reviews'],
                             'distance': similar_item_ids[1]}).reset_index(drop=True)

print(f"Reviews post: '{df.iloc[example_id]['reviews']}'\nNearest neighbors:")
results



Reviews post: 'Very aggravating....will not track items when you need it most. Mine will not track my steps when I'm at the end of an award based on steps. At other times you can't input your data for the day. This app is tied to my insurance for wellness program points and is about to make me miss out on points.'
Nearest neighbors:


,reviews,distance


**2- Given a query that we write, retrieve the most similar posts**

In [57]:
# Paste your API key here. Remember to not share publicly
api_key = 'mZyhjGS0iHcDBWzy5kpfVQfJW8Ezeemra6kGUEQi'

# Create and retrieve a Cohere API key from os.cohere.ai
co = cohere.Client(api_key)

In [ ]:
# query = "Best app to track healthcare related stuff"

# # Get the query's embedding
# # We'll need to embed the query using the same model that embedded the archive
# # so the query and archive are using the same embedding space.
# query_embed = co.embed(texts=[query],
#                   model="small-20220425",
#                   truncate="RIGHT").embeddings

# # Retrieve the nearest neighbors
# similar_item_ids = search_index.get_nns_by_vector(query_embed[0],10,
#                                                 include_distances=True)
# # Format & print the results
# results = pd.DataFrame(data={'texts': df.iloc[similar_item_ids[0]]['title'],
#                              'distance': similar_item_ids[1]})
# print(f"Query:'{query}'\nNearest neighbors:")
# results

In [59]:
embeds.shape

(1494, 1)

In [64]:
# Select the 'reviews_embeds' column from the DataFrame
embeds = embeds['reviews_embeds'].values

# Preprocess the embeds list to ensure uniformity
processed_embeds = [np.array(embed) for embed in embeds]

# Convert the processed embeds list into a numpy array
embeds_array = np.array(processed_embeds)

# Apply UMAP on the embeds_array
reducer = umap.UMAP(n_neighbors=100)
umap_embeds = reducer.fit_transform(embeds_array)


In [25]:
umap_embeds.shape

(3436, 2)

In [27]:
df.shape

(1494, 1)

In [66]:

df['x'] = umap_embeds[:,0]
df['y'] = umap_embeds[:,1]

# Plot
chart = alt.Chart(df).mark_circle(size=60).encode(
    x=#'x',
    alt.X('x',
        scale=alt.Scale(zero=False),
        axis=alt.Axis(labels=False, ticks=False, domain=False)
    ),
    y=
    alt.Y('y',
        scale=alt.Scale(zero=False),
        axis=alt.Axis(labels=False, ticks=False, domain=False)
    ),
    tooltip=['reviews']
).configure(background="#FDF7F0"
).properties(
    width=700,
    height=400,
    title='Ask HN: top 3,000 posts'
)

chart.interactive()

alt.Chart(...)

4- Cluster the reviews to identify the major common themes
Let's proceed the cluster the embeddings using KMeans from scikit-learn.

In [ ]:
print(embeds)

In [ ]:
# # Convert the embeds array to a numeric format
# embeds = np.asarray(embeds)
# print(embeds)

In [77]:
import numpy as np
from sklearn.cluster import KMeans

# Pick the number of clusters
n_clusters = 5

# Convert the embeds array to a two-dimensional numeric format
embeds = np.array([np.array(embed) for embed in embeds])

# Cluster the embeddings
kmeans_model = KMeans(n_clusters=n_clusters, random_state=0)
classes = kmeans_model.fit_predict(embeds)




5- Extract major keywords from each cluster so we can identify what the cluster is about

In [80]:
from sklearn.feature_extraction.text import TfidfTransformer

In [ ]:
!pip install bertopic

In [ ]:
# from bertopic._ctfidf import ClassTFIDF

In [86]:
# Extract the keywords for each cluster
documents =  df['reviews']
documents = pd.DataFrame({"Document": documents,
                          "ID": range(len(documents)),
                          "Topic": None})
documents['Topic'] = classes
documents_per_topic = documents.groupby(['Topic'], as_index=False).agg({'Document': ' '.join})
count_vectorizer = CountVectorizer(stop_words="english").fit(documents_per_topic.Document)
count = count_vectorizer.transform(documents_per_topic.Document)
words = count_vectorizer.get_feature_names_out()
tfidf_transformer = TfidfTransformer()
ctfidf = tfidf_transformer.fit_transform(count).toarray()
words_per_class = {label: [words[index] for index in ctfidf[label].argsort()[-10:]] for label in documents_per_topic.Topic}
df['cluster'] = classes
df['keywords'] = df['cluster'].map(lambda topic_num: ", ".join(np.array(words_per_class[topic_num])[:]))

Plot with clusters and keywords information
We can now plot the documents with their clusters and keywords

In [88]:
selection = alt.selection_multi(fields=['keywords'], bind='legend')

chart = alt.Chart(df).transform_calculate(
    url='https://news.ycombinator.com/item?id=' + alt.datum.id
).mark_circle(size=60, stroke='#666', strokeWidth=1, opacity=0.3).encode(
    x=#'x',
    alt.X('x',
        scale=alt.Scale(zero=False),
        axis=alt.Axis(labels=False, ticks=False, domain=False)
    ),
    y=
    alt.Y('y',
        scale=alt.Scale(zero=False),
        axis=alt.Axis(labels=False, ticks=False, domain=False)
    ),
    href='url:N',
    color=alt.Color('keywords:N',
                    legend=alt.Legend(columns=1, symbolLimit=0, labelFontSize=14)
                   ),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.2)),
    tooltip=['reviews', 'keywords']
).properties(
    width=800,
    height=500
).add_selection(
    selection
).configure_legend(labelLimit= 0).configure_view(
    strokeWidth=0
).configure(background="#FDF7F0").properties(
    title='Ask HN: Top 3,000 Posts'
)
chart.interactive()

alt.Chart(...)